# Anomaly Detection Project

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from env import user, host, password
import os
import wrangle as w

# Acquire and Prep

In [2]:
df = w.get_cohort_logs_data()
# get data

In [3]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [4]:
df.info()
# drop 'deleted_at'
# rows that didn't have a 'cohort_id' have nulls

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 109.9+ MB


In [5]:
#convert to object:
# user_id, cohort_id
df['user_id'] = df.user_id.astype(object)
df['cohort_id'] = df.cohort_id.astype(object)

In [6]:
df.cohort_id.dtype

dtype('O')

In [7]:
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [8]:
no_cid = df[df.cohort_id.isna()]
#selects rows without a cohort_id

In [9]:
no_cid.user_id.unique()
# These users had access but were not linked to any cohort. 
# However they have to have had some some sort of codeup association

array([48, 54, 58, 59, 61, 62, 63, 73, 74, 78, 79, 86, 88, 89, 100, 103,
       111, 137, 166, 176, 213, 247, 317, 349, 350, 351, 352, 353, 354,
       355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367,
       368, 369, 372, 375, 403, 406, 429, 544, 64, 713, 714, 715, 716,
       717, 718, 663, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728,
       729, 346, 731, 736, 644, 744, 782, 810, 814, 815], dtype=object)

In [10]:

for i in no_cid.user_id.unique(): #looks at the list of users who had a NaN under 'cohort_id'
    if df[df.user_id == i].cohort_id.isna().mean() < 1: # Mean of NaN's present under 'cohort_id' column for users in above list
        print(i, df[df.user_id == i].cohort_id.isna().mean()) # print user and mean if less than 1
# list of users who have something other than NaN present under 'cohort_id' column.    


88 0.4190715181932246
358 0.11423841059602649
375 0.07478753541076487
64 0.0015293325992536857
663 0.013468013468013467
346 0.47548711502199875
644 0.0035863717872086074
744 0.09583333333333334


In [11]:
x = df[df.user_id == 88].cohort_id.unique().tolist()
type(x)

list

In [12]:
list_users_w_cid = [88, 358, 375, 64, 663, 346, 644, 744]

for i in list_users_w_cid:
    x = df[df.user_id == i].cohort_id.unique().tolist()
    print(f'User_id ={i} is linked to the following cohorts: {x}')
    print('------------------')
          

User_id =88 is linked to the following cohorts: [nan, 7.0, 12.0, 11.0]
------------------
User_id =358 is linked to the following cohorts: [nan, 34.0]
------------------
User_id =375 is linked to the following cohorts: [nan, 31.0]
------------------
User_id =64 is linked to the following cohorts: [1.0, 28.0, nan, 52.0]
------------------
User_id =663 is linked to the following cohorts: [58.0, nan, 134.0]
------------------
User_id =346 is linked to the following cohorts: [29.0, 21.0, nan]
------------------
User_id =644 is linked to the following cohorts: [57.0, nan]
------------------
User_id =744 is linked to the following cohorts: [nan, 28.0]
------------------


In [13]:
#The following users are linked to multiple cohorts:
#    88, 64, 663, 346
#Maybe they signed up to multiple cohorts
#
#The other users are only linked to one cohort:
#    358 = 34, 375 = 31, 644 = 57, 744 = 28
#Perhaps these students were allowed access before they were assigned a cohort.     


In [14]:
df.path.unique().shape
# 2314 unique paths

(2314,)

In [15]:
df.path.value_counts().head(20)
#look only at top 10 paths

/                                                                            50313
search/search_index.json                                                     19519
javascript-i                                                                 18983
toc                                                                          18297
java-iii                                                                     13733
html-css                                                                     13635
java-ii                                                                      12685
spring                                                                       12524
jquery                                                                       11525
mysql                                                                        11033
java-i                                                                       10865
javascript-ii                                                                10730
appe

In [16]:
df['path_1'] = df.path.str.split('/').str[0]

In [17]:
df['path_2'] = df.path.str.split('/').str[1]

In [18]:
df['path_3'] = df.path.str.split('/').str[2]

In [19]:
df['path_4'] = df.path.str.split('/').str[3]

In [20]:
df['path_5'] = df.path.str.split('/').str[4]

In [21]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path_1,path_2,path_3,path_4,path_5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,,,NaN,NaN,NaN
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,NaN,NaN,NaN,NaN
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,object-oriented-programming,NaN,NaN,NaN
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,slides,object_oriented_programming,NaN,NaN,NaN
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0,javascript-i,conditionals,NaN,NaN,NaN


In [22]:
df[df.path_5.isna() == False]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path_1,path_2,path_3,path_4,path_5
2071,2018-01-30,11:37:48,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2073,2018-01-30,11:37:49,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2075,2018-01-30,11:37:50,content/laravel/quickstart/gitbook/images/favi...,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,quickstart,gitbook,images
2078,2018-01-30,11:38:31,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2080,2018-01-30,11:38:33,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881764,2021-04-08,15:05:29,appendix/open_data/www.who.int/research/en,837,133.0,66.69.94.59,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,NaN,3.0,appendix,open_data,www.who.int,research,en
881780,2021-04-08,15:13:10,appendix/open_data/www.who.int/research/en,837,133.0,66.69.94.59,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,NaN,3.0,appendix,open_data,www.who.int,research,en
882420,2021-04-09,00:10:17,content/html-css/gitbook/images/favicon.ico,80,14.0,136.50.29.193,14.0,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,NaN,1.0,content,html-css,gitbook,images,favicon.ico
882426,2021-04-09,00:10:29,content/javascript/gitbook/images/favicon.ico,80,14.0,136.50.29.193,14.0,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,NaN,1.0,content,javascript,gitbook,images,favicon.ico


In [23]:
df.head()
#drop id, deleted_at

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path_1,path_2,path_3,path_4,path_5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,,,NaN,NaN,NaN
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,NaN,NaN,NaN,NaN
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,object-oriented-programming,NaN,NaN,NaN
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,slides,object_oriented_programming,NaN,NaN,NaN
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0,javascript-i,conditionals,NaN,NaN,NaN


In [24]:
df = df.drop(columns=['id', 'deleted_at'])
#id is that same as cohort_id, and 'deleted_at' only has NaN values

In [25]:
df.head()

,date,time,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,path_1,path_2,path_3,path_4,path_5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,,,NaN,NaN,NaN
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,java-ii,NaN,NaN,NaN,NaN
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,java-ii,object-oriented-programming,NaN,NaN,NaN
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,slides,object_oriented_programming,NaN,NaN,NaN
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,javascript-i,conditionals,NaN,NaN,NaN


In [80]:
df.path


0                                                        /
1                                                  java-ii
2                      java-ii/object-oriented-programming
3                       slides/object_oriented_programming
4                                javascript-i/conditionals
                                ...                       
900218                                jquery/personal-site
900219                                   jquery/mapbox-api
900220                             jquery/ajax/weather-map
900221    anomaly-detection/discrete-probabilistic-methods
900222                                   jquery/mapbox-api
Name: path, Length: 900223, dtype: object

0

# Answers to Questions

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

PHP Full Stack Web Development

In [53]:
df[df.program_id == 1].path.value_counts().head(15)

/                                                                            1681
index.html                                                                   1011
javascript-i                                                                  736
html-css                                                                      542
spring                                                                        501
java-iii                                                                      479
java-ii                                                                       454
java-i                                                                        444
javascript-ii                                                                 429
appendix                                                                      409
jquery                                                                        344
mysql                                                                         284
content/html-css

In [47]:
# "javascript-i/introduction/working-with-data-types-operators-and-variables" is the most viewed lesson
# The paths above appears to be modules, not lessons

Java Full Stack Web Development

In [55]:
df[df.program_id == 2].path.value_counts().head(15)

/                                                                            35814
javascript-i                                                                 17457
toc                                                                          17428
search/search_index.json                                                     15212
java-iii                                                                     12683
html-css                                                                     12569
java-ii                                                                      11719
spring                                                                       11376
jquery                                                                       10693
mysql                                                                        10318
java-i                                                                       10016
javascript-ii                                                                 9861
appe

In [56]:
# javascript-i/introduction/working-with-data-types-operators-and-variables is the most viewed lesson for this program
# the paths above it appear to be modules

Data Science Program

In [57]:
df[df.program_id == 3].path.value_counts().head(15)

/                                                    8358
search/search_index.json                             2203
classification/overview                              1785
1-fundamentals/modern-data-scientist.jpg             1655
1-fundamentals/AI-ML-DL-timeline.jpg                 1651
1-fundamentals/1.1-intro-to-data-science             1633
classification/scale_features_or_not.svg             1590
fundamentals/AI-ML-DL-timeline.jpg                   1443
fundamentals/modern-data-scientist.jpg               1438
sql/mysql-overview                                   1424
fundamentals/intro-to-data-science                   1413
6-regression/1-overview                              1124
anomaly-detection/AnomalyDetectionCartoon.jpeg        829
anomaly-detection/overview                            804
10-anomaly-detection/AnomalyDetectionCartoon.jpeg     754
Name: path, dtype: int64

In [58]:
# 'classification/overview' is the most viewed lesson for the data science program

Front End Web Dev

In [59]:
df[df.program_id == 4].path.value_counts().head(15)

content/html-css                               2
content/html-css/introduction.html             1
content/html-css/gitbook/images/favicon.ico    1
/                                              1
Name: path, dtype: int64

In [62]:
(df.program_id == 4).sum()

5

In [63]:
# The data only contains 5 rows pertaining to this program. content/html-css being the most viewed lesson

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [65]:
df.groupby('cohort_id').path.value_counts()

cohort_id  path                                       
1.0        /                                              626
           javascript-i                                   294
           html-css                                       215
           javascript-ii                                  204
           spring                                         192
                                                         ... 
139.0      java-iii/servlets                                1
           javascript-i/bom-and-dom/dom                     1
           javascript-i/objects                             1
           javascript-i/objects/math                        1
           jquery/essential-methods/attributes-and-css      1
Name: path, Length: 13565, dtype: int64

In [67]:
df[df.cohort_id == 1].path.value_counts().head()

/                626
javascript-i     294
html-css         215
javascript-ii    204
spring           192
Name: path, dtype: int64

In [ ]:
for i in df['cohort_id']:
    df[df.cohort_id == i].path.value_counts().head()
    